In [159]:
import re
import pandas as pd

In [160]:
# file = 'LocalSP2000/LocalmunSP2000.txt'
# regiao = 'municipio'

file = 'LocalSP2000/LocalSP2000.txt'
regiao = 'SP'

with open(f'data/{file}', 'r', errors='ignore') as f:
    data = f.readlines()

In [161]:
# removendo linhas em branco
data = list(filter(lambda linha: linha != '\n', data))

In [162]:
# removendo cabecalho
data = list(filter(lambda linha: not linha.startswith('TRIBUNAL REGIONAL ELEITORAL - TRE'), data))
data = list(filter(lambda linha: not linha.startswith('SECRETARIA DE INFORMATICA   - SI'), data))
data = list(filter(lambda linha: not linha.startswith('SISTEMA DE ALISTAMENTO ELEITORAL'), data))
data = list(filter(lambda linha: not linha.__contains__('LOCAIS DE VOTACAO'), data))
data = list(filter(lambda linha: linha != '------------------------------------------------------------------------------\n', data))


In [163]:
# removendo quebra de linha
data = list(map(lambda linha: linha[:-1], data))

In [164]:
data

['ZONA: 001     MUNICIPIO: 71072 - SAO PAULO',
 'LOCAL: 1015 - COLEGIO OBJETIVO',
 'END. : AV. PAULISTA, 900            BELA VISTA         000000  CENTRO         ',
 'CEP  : 0 No. POSSIVEL SECOES: 30 QTDE MAX. ELEITORES: 18000',
 '0001/0482   0002/0480   0003/0478   0004/0482   0005/0478   0006/0484',
 '0007/0478   0008/0485   0009/0481   0010/0480   0011/0478   0012/0479',
 '0013/0482   0014/0479   0015/0480   0016/0477   0017/0481   0018/0479',
 '0019/0480   0020/0481   0021/0482   0022/0477   0023/0478   0024/0481',
 '0025/0481   0026/0484   0027/0480   0355/0482   0361/0480',
 'TOTAL: 29 secoes / 13929 eleitores',
 'LOCAL: 1023 - EEPG RODRIGUES ALVES',
 'END. : AV. PAULISTA, 227            BELA VISTA         000000  CENTRO         ',
 'CEP  : 0 No. POSSIVEL SECOES: 23 QTDE MAX. ELEITORES: 13800',
 '0029/0467   0030/0466   0031/0469   0032/0467   0033/0470   0034/0469',
 '0035/0467   0036/0465   0037/0470   0038/0471   0039/0469   0040/0465',
 '0041/0467   0042/0464   0043/0464   00

In [165]:
# Iterando sobre cada linha

resultado = []
padrao_cep = re.compile("[0-9]{5}[-]{0,1}[0-9]{3}")
padrao_secao_eleitores = re.compile("[0-9]{4}[/][0-9]+")
valida_secoes, valida_eleitores = [], []

for i in data:
    if i.startswith('ZONA'):
        zona = i[6:9]
        cod_municipio = i.split(' - ')[0].split(': ')[-1]
        municipio = i.split(' - ')[-1]

    if i.startswith('LOCAL'):
        cod_local = i.split(': ')[-1].split('-')[0]
        local = i.split(': ')[-1].split('-')[1]

    if i.startswith('END'):
        endereco = i.split(':')[-1].lstrip()

    # if i.startswith('Bairro    :'):
    #     bairro = i.split(':')[-1].lstrip()

    if i.startswith('CEP'):
        cep = i.split(':')[1].lstrip().split(' ')[0]
        # valida_secoes.append(int(i.split(':')[-2].lstrip().split(' ')[0]))
        # valida_eleitores.append(int(i.split(':')[-1].lstrip()))
    else:
        secao_eleitores = padrao_secao_eleitores.findall(i)
        for s in secao_eleitores:
            secao = s.split('/')[0]
            eleitores = int(s.split('/')[1])
            resultado.append([cod_municipio, municipio, cod_local, local, endereco, cep, zona, secao, eleitores])

    if i.startswith('TOTAL:') and not i.__contains__('nenhuma secao encontrada'):
        valida_secoes.append(int(i.split(':')[-1].lstrip().split('/')[0].split(' ')[0]))
        valida_eleitores.append(int(i.split(':')[-1].lstrip().split('/')[1].lstrip().split(' ')[0]))

    # break


In [166]:
i

'Locais de Votacao: 7077     Secoes: 47398     Eleitores: 24263612'

In [167]:
zona

'416'

In [168]:
df_locais_votacao = pd.DataFrame(resultado, columns=['cod_municipio', 'municipio', 'cod_local', 'local', 'endereco', 'cep', 'zona', 'secao', 'eleitores'])

In [169]:
df_locais_votacao

,cod_municipio,municipio,cod_local,local,endereco,cep,zona,secao,eleitores
0,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",0,001,0001,482
1,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",0,001,0002,480
2,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",0,001,0003,478
3,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",0,001,0004,482
4,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",0,001,0005,478
...,...,...,...,...,...,...,...,...,...
47394,71579,TABOAO DA SERRA,1163,EEPG.REVERENDO NICODEMOS ELLER,"RUA LUIZ QUEIROZ, 338 ...",6785040,416,0133,111
47395,71579,TABOAO DA SERRA,1171,EEPSG. GILBERTO FREIRE,"RUA MANOEL LEITE DA CUNHA, 590 ...",6773140,416,0121,600
47396,71579,TABOAO DA SERRA,1171,EEPSG. GILBERTO FREIRE,"RUA MANOEL LEITE DA CUNHA, 590 ...",6773140,416,0124,600
47397,71579,TABOAO DA SERRA,1171,EEPSG. GILBERTO FREIRE,"RUA MANOEL LEITE DA CUNHA, 590 ...",6773140,416,0127,600


In [170]:
sum(valida_secoes)

47395

In [171]:
sum(valida_eleitores)

24263612

In [172]:
df_locais_votacao.eleitores.sum()

24265445

In [173]:
df_locais_votacao.to_csv(f'resultados/locais_de_votacao-2000-{regiao}.csv')